# Introduction to Data Science
## Model assessment

Import all of the packages we will need.

In [ ]:
# Import the libraries we will be using
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8

model_types = ["logistic_regression", "decision_tree"]

### Data
We're going to use a mail response data set from a real direct marketing campaign located in `data/mailing.csv`. Each record represents an individual who was targeted with a direct marketing offer.  The offer was a solicitation to make a charitable donation. 

The columns (features) are:

```
income       household income
Firstdate    data assoc. with the first gift by this individual
Lastdate     data associated with the most recent gift 
Amount       average amount by this individual over all periods (incl. zeros)
rfaf2        frequency code
rfaa2        donation amount code
pepstrfl     flag indicating a star donator
glast        amount of last gift
gavr         amount of average gift
```

The target variables is `class` and is equal to one if they gave in this campaign and zero otherwise.

In [ ]:
# Load the data
data = pd.read_csv("data/mailing.csv")

In [ ]:
# Let's take a look at the data
data.head()

From the description above, and the head of the data, we see that two of the fields are **categorical** (text) instead of the typical **numerical** fields we have been looking at until this point. Today, one of the models we will be using is a logistic regression. From the previous classes, we have seen that logistic regression requires *all* fields to be numerical. To do this, we are going to create dummy variables for all the fields that are categorical.

#### Dummyize
The typical way to create dummys for a field is to create new variables for each possible category of the field. For example consider a field called color that can have the possible values "red", "blue", and "green". To dummyize color, we would create three new features: "color_red", "color_blue", and "color_green". These fields would take the value 1 or 0 depending on the actual value of color. Each record can only have one of these fields set to 1!

As a small detail, you should leave out one of the possible categories. So, for example, in the above example that had three possible values, you should only create two dummies. The value you leave out doesn't matter, you can choose it at random.

Let's dummyize the fields `rfaa2` and `pepstrfl`.

In [ ]:
for field in ['rfaa2', 'pepstrfl']:
    # Go through each possible value except the last one
    for value in data[field].unique()[0:-1]:
        # Create a new binary field
        data[field + "_" + value] = pd.Series(data[field] == value, dtype=int)

    # Drop the original field
    data = data.drop([field], axis=1)

In [ ]:
# Let's take another look at the data
data.head()

Everything is now numeric! Let's wrap up by putting all of the *predictors* into `X` and the target variable into `Y`. We will also split our data into training and test sets.

In [ ]:
X = data.drop(['class'], axis=1)
Y = data['class']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.75)

### Confusion matrices
Let's build a confusion matrix using a logistic regression model with the default setting of predicting a 1 if the probability is $\geq$ 50%.

Remember, a confusion matrix looks like:

```
  |____________ p __________|___________ n ___________|
Y | 1's predicted to be 1's | 0's predicted to be 1's |
N | 1's predicted to be 0's | 0's predicted to be 0's |
```

In [ ]:
# Make and fit a model on the training data
model = LogisticRegression(C=1000000)
model.fit(X_train, Y_train)

# Get probabilities of being a 1
probabilities = model.predict_proba(X_test)[:, 1]

In [ ]:
# Use the threshold of 0.5 to predict a 1
prediction = probabilities > 0.5

# Build and print a confusion matrix
confusion_matrix_large = pd.DataFrame(metrics.confusion_matrix(Y_test, prediction, labels=[1, 0]).T,
                                columns=['p', 'n'], index=['Y', 'N'])
print confusion_matrix_large

What if we lower the threshold to 5%?

In [ ]:
# Let's move the threshold down
prediction = probabilities > 0.05

# Build and print a confusion matrix
confusion_matrix_small = pd.DataFrame(metrics.confusion_matrix(Y_test, prediction, labels=[1, 0]).T,
                                columns=['p', 'n'], index=['Y', 'N'])
print confusion_matrix_small

Is this good performance? How can we tell. Let's computer the profit it would produce.

### Profit curves
Let's say that our profit margin is small: each offer costs \$1 to make and market, and each accepted offer earns \$18, for a profit of $17. The cost matrix would be:

In [ ]:
cost_matrix = pd.DataFrame([[None, None], [None, None]], columns=['p', 'n'], index=['Y', 'N'])
print cost_matrix

The expected profit of using 50% and 5% as your prediction threshold would be

In [ ]:
print "Expected profit with a cutoff of 50%% is $%.2f." % np.sum(np.sum(confusion_matrix_large/float(len(Y_test)) * cost_matrix))
print "Expected profit with a cutoff of 5%% is $%.2f." % np.sum(np.sum(confusion_matrix_small/float(len(Y_test)) * cost_matrix))

If we try all possible thresholds, we can plot the profit curve. Note that here the total profit calculation implies some particular size of the population to which you would apply the model. Let's start by assuming that it is the same size as the test set.

In [ ]:
# Get the false positive rate, true positive rate, and all thresholds
fpr, tpr, thresholds = metrics.roc_curve(Y_test, model.predict_proba(X_test)[:,1])
Size_targeted_pop = float(len(Y_test))

# What is the baseline probability of being positive or negative in the data set?
p_p = np.sum(Y_test)/float(len(Y_test))
p_n = 1 - np.sum(Y_test)/float(len(Y_test))

# How many users are above the current threshold?
n_targeted = []
for t in thresholds:
    n_targeted.append(np.sum(probabilities >= t))

# Turn these counts to percentages of users above the threshold
n_targeted = np.array(n_targeted)/float(len(Y_test))

# Expected profits:  
expected_profits = (cost_matrix['p']['Y']*(tpr*p_p)) + (cost_matrix['n']['Y']*(fpr*p_n))

# Plot the profit curve
plt.plot(n_targeted, Size_targeted_pop*expected_profits)
plt.xlabel("Percentage of users targeted")
plt.ylabel("Profit")
plt.title("Profits")

### Cumulative response and lift curves
What if we can't specify the costs and benefits? Let's take a look at a cumulative response curve for two different classifiers.

In [ ]:
# Keep track of all output
xs = {}
ys = {}

for model_type in model_types:
    # Instantiate the model
    if model_type == "decision_tree":
        model = DecisionTreeClassifier(criterion="entropy", max_depth=15)
    elif model_type == "logistic_regression":
        model = LogisticRegression(C=1000000)
    model.fit(X_train, Y_train)

    # Get the predicted value and the probability of Y_test records being = 1
    Y_test_predicted = model.predict(X_test)
    Y_test_probability_1 = model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_test_probability_1)

    # How many users are above the current threshold?
    n_targeted = []
    for t in thresholds:
        n_targeted.append(np.sum(probabilities >= t))

    # Turn these counts to percentages of users above the threshold
    n_targeted = np.array(n_targeted)/float(len(Y_test))

    # Store
    xs[model_type] = n_targeted * 100
    ys[model_type] = tpr * 100
    
    # Plot
    plt.plot(n_targeted * 100, tpr * 100, label=model_type)# * np.sum(Y_test)/float(len(Y_test)))

plt.plot([0,100], [0,100], 'k--', label="Random")
plt.xlabel("Percentage of test instances targeted (decreasing score)")
plt.ylabel("Percentage of positives targeted")
plt.title("Cumulative response curve")
plt.legend(loc=2)
plt.show()

We can also easily look at a lift curve in this scenario.

In [ ]:
for model_type in model_types:
    x_lift = xs[model_type]
    y_lift = ys[model_type]/x_lift

    plt.plot(x_lift, y_lift, label=model_type)

plt.plot([0,100], [1,1], 'k--', label="Random")
plt.xlabel("Percentage of test instances targeted (decreasing score)")
plt.ylabel("Percentage of positives targeted")
plt.title("Lift curve")
plt.xlim([0,100])
plt.ylim([0,10])
plt.legend()
plt.show()

### Receiver operating characteristic (ROC) curves
For this data set, let's take a look at the ROC curves generated by our classifiers. This can be useful if you don't know the class distribution in the population.

In [ ]:
for model_type in model_types:
    # Instantiate the model
    if model_type == "decision_tree":
        model = DecisionTreeClassifier(criterion="entropy", max_depth=15)
    elif model_type == "logistic_regression":
        model = LogisticRegression(C=1000000)
    model.fit(X_train, Y_train)

    # Get the probability of Y_test records being = 1
    Y_test_probability_1 = model.predict_proba(X_test)[:, 1]

    # Use the metrics.roc_curve function to get the true positive rate (tpr) and false positive rate (fpr)
    fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_test_probability_1)

    # Get the area under the curve (AUC)
    acc = np.mean(cross_validation.cross_val_score(model, X, Y, scoring="accuracy"))
    auc = np.mean(cross_validation.cross_val_score(model, X, Y, scoring="roc_auc"))

    # Plot the ROC curve
    plt.plot(fpr, tpr, label=model_type + " (AUC = " + str(round(auc, 2)) + ")")
plt.xlabel("False positive rate (fpr)")
plt.ylabel("True positive rate (tpr)")
plt.plot([0,1], [0,1], 'k--', label="Random")
plt.legend(loc=2)